In [ ]:
# %%
"""
Dedalus script simulating 2D horizontally-periodic Rayleigh-Benard convection.
This script demonstrates solving a 2D Cartesian initial value problem. It can
be ran serially or in parallel, and uses the built-in analysis framework to save
data snapshots to HDF5 files. The `plot_snapshots.py` script can be used to
produce plots from the saved data. It should take about 5 cpu-minutes to run.

For incompressible hydro with two boundaries, we need two tau terms for each the
velocity and buoyancy. Here we choose to use a first-order formulation, putting
one tau term each on auxiliary first-order gradient variables and the others in
the PDE, and lifting them all to the first derivative basis. This formulation puts
a tau term in the divergence constraint, as required for this geometry.

To run and plot using e.g. 4 processes:
    $ mpiexec -n 4 python3 rayleigh_benard.py
    $ mpiexec -n 4 python3 plot_snapshots.py snapshots/*.h5
"""


# %%
import numpy as np
import dedalus.public as d3
import logging
logger = logging.getLogger(__name__)
import copy
import h5py
import numpy as np
import matplotlib
import re

import matplotlib.pyplot as plt
from dedalus.extras import plot_tools

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize

import os
from os import listdir

# save_dir= "/scratch/zb2113/DedalusData/4e50.33Q=0.0028"

#if not os.path.exists(save_dir):
#    os.mkdir(save_dir)

# %%
# Parameters
Lx, Lz = 64,1
Nx, Nz = 4096, 64
Ra_M = 4e5
D_0 = 0
D_H = 1/3
M_0 = 0
M_H = -1
N_s2=4/3
Qrad=0.0028

Prandtl = 1
dealias = 3/2
stop_sim_time = 1500
timestepper = d3.RK222
max_timestep = 0.125
dtype = np.float64
savefreq = 5

# The code check for the existence of a MRBC2D_param.py and import it. This allows yo update the simulations parameters

if ( os.path.isfile('./MRBC2D_param.py')):
    from MRBC2D_param import *

print(Ra_M)
# %%
# Bases
coords = d3.CartesianCoordinates('x','z')
dist = d3.Distributor(coords, dtype=dtype)
xbasis = d3.RealFourier(coords['x'], size=Nx, bounds=(0, Lx), dealias=dealias)
zbasis = d3.ChebyshevT(coords['z'], size=Nz, bounds=(0, Lz), dealias=dealias)

# Fields
p = dist.Field(name='p', bases=(xbasis,zbasis))
D = dist.Field(name='D', bases=(xbasis,zbasis))
M = dist.Field(name='M', bases=(xbasis,zbasis))
u = dist.VectorField(coords, name='u', bases=(xbasis,zbasis))
Z = dist.Field(name='Z', bases=zbasis)
T = dist.Field(name='T', bases=(xbasis,zbasis))
C = dist.Field(name='C', bases=(xbasis,zbasis))
Qr = dist.Field(name='Q', bases=(xbasis,zbasis))
Nsz = dist.Field(name='Nsz', bases=(xbasis,zbasis))

tau_p = dist.Field(name='tau_p')
tau_B1 = dist.Field(name='tau_B1', bases=xbasis)
tau_B2 = dist.Field(name='tau_B2', bases=xbasis)
tau_D1 = dist.Field(name='tau_D1', bases=xbasis)
tau_D2 = dist.Field(name='tau_D2', bases=xbasis)
tau_M1 = dist.Field(name='tau_M1', bases=xbasis)
tau_M2 = dist.Field(name='tau_M2', bases=xbasis)
tau_u1 = dist.VectorField(coords,name='tau_u1', bases=xbasis)
tau_u2 = dist.VectorField(coords,name='tau_u2', bases=xbasis)
tau_u3 = dist.Field(name='tau_u3', bases=xbasis)
tau_u4 = dist.Field(name='tau_u4', bases=xbasis)
tau_T1 = dist.Field(name='tau_t1', bases=xbasis)
tau_T2 = dist.Field(name='tau_t2', bases=xbasis)
tau_C1 = dist.Field(name='tau_c1', bases=xbasis)
tau_C2 = dist.Field(name='tau_c2', bases=xbasis)

# Substitutions    
#Kuo_Bretherton Equilibrium
kappa = (Ra_M * Prandtl/((M_0-M_H)*Lz**3))**(-1/2)
nu = (Ra_M / (Prandtl*(M_0-M_H)*Lz**3))**(-1/2)
print('kappa',kappa)
print('nu',nu)
print('RaM',Ra_M)
print('Qrad',Qrad)


x,z = dist.local_grids(xbasis,zbasis)
Z['g']=z
Qr['g']=np.sin(np.pi*z/Lz)
Nsz['g']=N_s2*z

ex,ez = coords.unit_vector_fields(dist)
lift_basis = zbasis.derivative_basis(1)
lift = lambda A: d3.Lift(A, lift_basis, -1)

B_op = (np.absolute(D - M - Nsz)+ M + D - Nsz)/2
lq = B_op/2 + np.absolute(B_op)



Max = lambda A,B: (abs(A-N_s2*Z-B)+A-N_s2*Z+B)/2
eva = lambda A: A.evaluate()

dz= lambda A: d3.Differentiate(A, coords['z'])
dx= lambda A: d3.Differentiate(A, coords['x'])


ux=u@ex
uz=u@ez
dxux=dx(ux)
dzux=dz(ux)
dxuz=dx(uz)
dzuz=dz(uz)

dzlq = d3.Differentiate(lq, coords['z'])
dzD = d3.Differentiate(D, coords['z'])
dzM = d3.Differentiate(M, coords['z'])
dzC = d3.Differentiate(C, coords['z'])
dzT = d3.Differentiate(T, coords['z'])

grad_u = d3.grad(u) + ez* lift(tau_u1) # First-order reduction
grad_ux = grad_u@ex # First-order reduction
grad_uz = grad_u@ez # First-order reduction
grad_M = d3.grad(M) + ez*lift(tau_M1) # First-order reduction
grad_D = d3.grad(D) + ez*lift(tau_D1) # First-order reduction
grad_T = d3.grad(T) + ez*lift(tau_T1)
grad_C = d3.grad(C) + ez*lift(tau_C1)

# Problem
# First-order form: "div(f)" becomes "trace(grad_f)"
# First-order form: "lap(f)" becomes "div(grad_f)"
problem = d3.IVP([p, M, D, u, T, C, tau_p, tau_M1, tau_M2, tau_D1, tau_D2, tau_u1, tau_u2, tau_T1, tau_T2, tau_C1, tau_C2], namespace=locals())
problem.add_equation("trace(grad_u) + tau_p= 0")
problem.add_equation("dt(M) - kappa*div(grad_M) + lift(tau_M2) = - u@grad(M)-Qrad/2*Qr")
problem.add_equation("dt(D) - kappa*div(grad_D) + lift(tau_D2) = - u@grad(D)-Qrad*Qr")
problem.add_equation("dt(u) - nu*div(grad_u) + grad(p)  + lift(tau_u2) = - u@grad(u)+ B_op*ez")
problem.add_equation("dt(T) - kappa*div(grad_T) + lift(tau_T2) = - u@grad(T)")
problem.add_equation("dt(C) - kappa*div(grad_C) + lift(tau_C2) = - u@grad(C)+1")
problem.add_equation("u(z=0) = 0")
problem.add_equation("uz(z=Lz) = 0")
problem.add_equation("dz(ux)(z=Lz)=0")
problem.add_equation("M(z=0) = M_0")
problem.add_equation("D(z=0) = D_0")
problem.add_equation("M(z=Lz) = M_H")
problem.add_equation("D(z=Lz) = D_H")
problem.add_equation("T(z=0) = 0")
problem.add_equation("T(z=1) = 1")
problem.add_equation("C(z=0) = 0")
problem.add_equation("dz(C)(z=Lz) = 0")
problem.add_equation("integ(p) = 0") # Pressure gauge

# %%
# Solver
solver = problem.build_solver(timestepper)
solver.stop_sim_time = stop_sim_time


# %%
# Initial condition
D.fill_random('g', seed=42, distribution='normal', scale=1e-3) # Random noise
D['g'] *= z * (Lz - z) # Damp noise at walls
D['g'] += (D_H-D_0)*z # Add linear background
M.fill_random('g', seed=28, distribution='normal', scale=1e-3) # Random noise
M['g'] *= z * (Lz - z) # Damp noise at walls
M['g'] += (M_H-M_0)*z # Add linear background
# T.fill_random('g', seed=42, distribution='normal', scale=1)


# %%
# Analysis
snapshots = solver.evaluator.add_file_handler('./snapshots', sim_dt=1, max_writes=500)
snapshots.add_task(d3.Average(dz(D),coords['x']),name='Dry derivative')
snapshots.add_task(d3.Average(dz(M),coords['x']),name='Moist derivative')
snapshots.add_task(-Lz*(2*dz(M)(z=0)-dz(D)(z=0))/(2*(M_0-M_H)-(D_0-D_H)),name='Nusselt Number')
snapshots.add_task(d3.Average(M, coords['x']), name='horizontal avg M')
snapshots.add_task(d3.Average(D, coords['x']), name='horizontal avg D')
snapshots.add_task(d3.Average(B_op, coords['x']), name='horizontal avg B')
snapshots.add_task(d3.Average(lq, coords['x']), name='horizontal avg liquid')
snapshots.add_task(d3.Average(T, coords['x']), name='horizontal avg T')
snapshots.add_task(d3.Average(C, coords['x']), name='horizontal avg C')
snapshots.add_task(d3.Average(uz, coords['x']), name='horizontal avg uz')
snapshots.add_task(d3.Average(uz*T, coords['x']), name='horizontal avg vertical T flux')
snapshots.add_task(d3.Average(uz*C, coords['x']), name='horizontal avg vertical C flux')
snapshots.add_task(d3.Average(uz*M, coords['x']), name='horizontal avg vertical M flux')
snapshots.add_task(d3.Average(uz*D, coords['x']), name='horizontal avg vertical D flux')
snapshots.add_task(d3.Average(uz*lq, coords['x']), name='horizontal avg vertical lq flux')
snapshots.add_task(d3.Integrate(0.5*u@u,coords),name='total KE')
# snapshots.add_task(uz, name='uz')
snapshots.add_task( - kappa * d3.Average(dzlq,'x'),layout='g', name='diffusive flux of water')
snapshots.add_task( - kappa * d3.Average(dzD,'x'),layout='g', name='diffusive flux of dry buoyancy')
snapshots.add_task( - kappa * d3.Average(dzM,'x'),layout='g', name='diffusive flux of moist buoyancy')
snapshots.add_task( - kappa * d3.Average(dzC,'x'),layout='g', name='diffusive flux of clock tracer')
snapshots.add_task( - kappa * d3.Average(dzT,'x'),layout='g', name='diffusive flux of tracer')
snapshots.add_task( d3.Average(uz*D,'x') - d3.Average(uz, coords['x'])*d3.Average(D, coords['x']),layout='g', name='Reynolds flux of dry buoyancy')
snapshots.add_task( d3.Average(uz*M,'x') - d3.Average(uz, coords['x'])*d3.Average(M, coords['x']),layout='g', name='Reynolds flux of moist buoyancy')
snapshots.add_task( d3.Average(uz*C,'x') - d3.Average(uz, coords['x'])*d3.Average(C, coords['x']),layout='g', name='Reynolds flux of clock tracer')
snapshots.add_task( d3.Average(uz*T,'x') - d3.Average(uz, coords['x'])*d3.Average(T, coords['x']),layout='g', name='Reynolds flux of tracer')

restart = solver.evaluator.add_file_handler('./restart', sim_dt=500.0, max_writes=1)
restart.add_tasks(solver.state)

analysis = solver.evaluator.add_file_handler('./analysis', sim_dt=savefreq, max_writes=1)
analysis.add_task(D, name='D')
analysis.add_task(M, name='M')
analysis.add_task(C, name='C')
analysis.add_task(T, name='T')
analysis.add_task(uz, name='uz')




# %%
# CFL
CFL = d3.CFL(solver, initial_dt=0.1, cadence=1, safety=0.3, threshold=0.05,
             max_change=1.1, min_change=0.25, max_dt=max_timestep)
CFL.add_velocity(u)

# %%
# Flow properties
flow = d3.GlobalFlowProperty(solver, cadence=10)
flow.add_property(np.sqrt(u@u)/nu, name='Re')


# %%
# Main loop
startup_iter = 10
try:
    logger.info('Starting main loop')
    while solver.proceed:
        timestep = CFL.compute_timestep()
        solver.step(timestep)
        if (solver.iteration-1) % 10 == 0:
            max_Re = flow.max('Re')
            logger.info('Iteration=%i, Time=%e, dt=%e, max(Re)=%f' %(solver.iteration, solver.sim_time, timestep, max_Re))
except:
    logger.error('Exception raised, triggering end of main loop.')
    raise
finally:
    solver.log_stats()



400000.0
kappa 0.0015811388300841897
nu 0.0015811388300841897
RaM 400000.0
Qrad 0.0028
2024-08-05 21:10:21,781 subsystems 0/1 INFO :: Building subproblem matrices 1/2048 (~0%) Elapsed: 0s, Remaining: 7m 54s, Rate: 4.3e+00/s
2024-08-05 21:10:31,734 subsystems 0/1 INFO :: Building subproblem matrices 84/2048 (~4%) Elapsed: 10s, Remaining: 3m 58s, Rate: 8.2e+00/s
2024-08-05 21:10:41,755 subsystems 0/1 INFO :: Building subproblem matrices 168/2048 (~8%) Elapsed: 20s, Remaining: 3m 46s, Rate: 8.3e+00/s
2024-08-05 21:10:46,033 subsystems 0/1 INFO :: Building subproblem matrices 205/2048 (~10%) Elapsed: 24s, Remaining: 3m 40s, Rate: 8.4e+00/s
2024-08-05 21:10:51,605 subsystems 0/1 INFO :: Building subproblem matrices 252/2048 (~12%) Elapsed: 30s, Remaining: 3m 34s, Rate: 8.4e+00/s
2024-08-05 21:11:01,602 subsystems 0/1 INFO :: Building subproblem matrices 335/2048 (~16%) Elapsed: 40s, Remaining: 3m 25s, Rate: 8.4e+00/s
2024-08-05 21:11:10,273 subsystems 0/1 INFO :: Building subproblem matrice